# Time Series Analysis

In [8]:
from pandas_datareader import data, wb
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import pandas as pd
import datetime as dt
import urllib.request, json 
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
import pandas_summary
import quandl
import datetime
import fbprophet
%matplotlib inline
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)
pd.set_option('precision', 3)

## Get Historical Data from Quandl

In [109]:
quandl.ApiConfig.api_key = 'ZNB9kQtgKML6KWfbbMNN'

# Retrieve STOCKS data from Quandl
amzn_df = quandl.get('WIKI/AMZN')
aapl_df = quandl.get('WIKI/AAPL')
msft_df = quandl.get('WIKI/MSFT')
googl_df = quandl.get('WIKI/GOOGL')
ba_df = quandl.get('WIKI/BA')
aal_df = quandl.get('WIKI/AAL')

In [113]:
quandl.ApiConfig.api_key = 'ZNB9kQtgKML6KWfbbMNN'
amzn = quandl.get('WIKI/AMZN', start_date="2001-12-31")

In [4]:
amzn_df.to_csv('./data/amazon')
aapl_df.to_csv('./data/apple')
msft_df.to_csv('./data/microsoft')
ba_df.to_csv('./data/boeing')
aal_df.to_csv('./data/aairline')

In [6]:
amzn = pd.read_csv('./data/amazon', index_col=0, parse_dates=True)
aapl = pd.read_csv('./data/apple', index_col=0, parse_dates=True)
msft = pd.read_csv('./data/microsoft', index_col=0, parse_dates=True)
ba = pd.read_csv('./data/boeing', index_col=0, parse_dates=True)
aal = pd.read_csv('./data/aairline', index_col=0, parse_dates=True)

In [7]:
msft[:3]

            Open   High   Low  Close     Volume  Ex-Dividend  Split Ratio  \
Date                                                                        
1986-03-13  25.5  29.25  25.5   28.0  3.583e+06          0.0          1.0   
1986-03-14  28.0  29.50  28.0   29.0  1.070e+06          0.0          1.0   
1986-03-17  29.0  29.75  29.0   29.5  4.624e+05          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
1986-03-13      0.059      0.068     0.059       0.065    1.032e+09  
1986-03-14      0.065      0.068     0.065       0.067    3.082e+08  
1986-03-17      0.067      0.069     0.067       0.068    1.332e+08  

In [20]:
aapl[:3]

             Open   High    Low  Close     Volume  Ex-Dividend  Split Ratio  \
Date                                                                          
1980-12-12  28.75  28.87  28.75  28.75  2.094e+06          0.0          1.0   
1980-12-15  27.38  27.38  27.25  27.25  7.852e+05          0.0          1.0   
1980-12-16  25.37  25.37  25.25  25.25  4.720e+05          0.0          1.0   

            Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
Date                                                                 
1980-12-12      0.423      0.424     0.423       0.423    1.173e+08  
1980-12-15      0.403      0.403     0.401       0.401    4.397e+07  
1980-12-16      0.373      0.373     0.371       0.371    2.643e+07  

In [28]:
msftAV = msft[['Adj. Close', 'Volume']]
aaplAV = aapl[['Adj. Close', 'Volume']]
pd.concat([msftAV, aapllAV])

            Adj. Close     Volume
Date                             
1986-03-13       0.065  3.583e+06
1986-03-14       0.067  1.070e+06
1986-03-17       0.068  4.624e+05
1986-03-18       0.066  2.353e+05
...                ...        ...
2018-03-22     168.845  4.105e+07
2018-03-23     164.940  4.025e+07
2018-03-26     172.770  3.627e+07
2018-03-27     168.340  3.896e+07

[17476 rows x 2 columns]

In [23]:
aaplA = aapl[['Adj. Close']]
pd.concat([msftAV, aaplA], sort=True)

            Adj. Close     Volume
Date                             
1986-03-13       0.065  3.583e+06
1986-03-14       0.067  1.070e+06
1986-03-17       0.068  4.624e+05
1986-03-18       0.066  2.353e+05
...                ...        ...
2018-03-22     168.845        NaN
2018-03-23     164.940        NaN
2018-03-26     172.770        NaN
2018-03-27     168.340        NaN

[17476 rows x 2 columns]

In [24]:
pd.concat([msftAV, aaplA], join='inner')

            Adj. Close
Date                  
1986-03-13       0.065
1986-03-14       0.067
1986-03-17       0.068
1986-03-18       0.066
...                ...
2018-03-22     168.845
2018-03-23     164.940
2018-03-26     172.770
2018-03-27     168.340

[17476 rows x 1 columns]

In [26]:
msftA = msft[['Adj. Close']]
closes = pd.concat([msftA, aaplA], axis=1)
closes[:3]

            Adj. Close  Adj. Close
Date                              
1980-12-12         NaN       0.423
1980-12-15         NaN       0.401
1980-12-16         NaN       0.371

In [31]:
pd.concat([msftAV[:5], aaplAV[:3]], axis=1,
         keys=['MSFT','AAPL'])

                 MSFT                  AAPL           
           Adj. Close     Volume Adj. Close     Volume
Date                                                  
1980-12-12        NaN        NaN      0.423  2.094e+06
1980-12-15        NaN        NaN      0.401  7.852e+05
1980-12-16        NaN        NaN      0.371  4.720e+05
1986-03-13      0.065  3.583e+06        NaN        NaN
1986-03-14      0.067  1.070e+06        NaN        NaN
1986-03-17      0.068  4.624e+05        NaN        NaN
1986-03-18      0.066  2.353e+05        NaN        NaN
1986-03-19      0.065  1.663e+05        NaN        NaN

In [33]:
pd.concat([msftA[:5], aaplA[:3]], axis=1, join='inner',keys=['MSFT','AAPL'])

Empty DataFrame
Columns: [(MSFT, Adj. Close), (AAPL, Adj. Close)]
Index: []

In [34]:
pd.concat([msftA[:3], aaplA[:3]], ignore_index=True)

   Adj. Close
0       0.065
1       0.067
2       0.068
3       0.423
4       0.401
5       0.371

In [35]:
msftAR = msftA.reset_index()
msftVR = msft[['Volume']].reset_index()
msftAR[:3]

        Date  Adj. Close
0 1986-03-13       0.065
1 1986-03-14       0.067
2 1986-03-17       0.068

In [36]:
msftVR[:3]

        Date     Volume
0 1986-03-13  3.583e+06
1 1986-03-14  1.070e+06
2 1986-03-17  4.624e+05

In [37]:
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

        Date  Adj. Close     Volume
0 1986-03-13       0.065  3.583e+06
1 1986-03-14       0.067  1.070e+06
2 1986-03-17       0.068  4.624e+05
3 1986-03-18       0.066  2.353e+05
4 1986-03-19       0.065  1.663e+05

In [38]:
msftAR0_5 = msftAR[0:5]
msftAR0_5

        Date  Adj. Close
0 1986-03-13       0.065
1 1986-03-14       0.067
2 1986-03-17       0.068
3 1986-03-18       0.066
4 1986-03-19       0.065

In [39]:
msftVR2_4 = msftVR[2:4]
msftVR2_4

        Date    Volume
2 1986-03-17  462400.0
3 1986-03-18  235300.0

In [40]:
# Inner Join - Merge is defaulted to inner join and output shows
# only the 2 dates that was a match between Adj Close and Vol b/w MS and AP
pd.merge(msftAR0_5, msftVR2_4)

        Date  Adj. Close    Volume
0 1986-03-17       0.068  462400.0
1 1986-03-18       0.066  235300.0

In [42]:
#This can be changed to an outer join with how='outer'. All rows from the outer
#DataFrame are returned (msftAR0_5), and values not found in the inner DataFrame
#(msftVR2_4) are replaced with NaN:
pd.merge(msftAR0_5, msftVR2_4, how='outer')

        Date  Adj. Close    Volume
0 1986-03-13       0.065       NaN
1 1986-03-14       0.067       NaN
2 1986-03-17       0.068  462400.0
3 1986-03-18       0.066  235300.0
4 1986-03-19       0.065       NaN

In [46]:
msft.insert(0, 'Symbol', 'MSFT')
aapl.insert(0, 'Symbol', 'AAPL')
combined = pd.concat([msft, aapl]).sort_index()
s4p = combined.reset_index();

ValueError: cannot insert Symbol, already exists

In [47]:
s4p[:5]

        Date Symbol   Open   High    Low  Close     Volume  Ex-Dividend  \
0 1980-12-12   AAPL  28.75  28.87  28.75  28.75  2.094e+06          0.0   
1 1980-12-15   AAPL  27.38  27.38  27.25  27.25  7.852e+05          0.0   
2 1980-12-16   AAPL  25.37  25.37  25.25  25.25  4.720e+05          0.0   
3 1980-12-17   AAPL  25.87  26.00  25.87  25.87  3.859e+05          0.0   
4 1980-12-18   AAPL  26.63  26.75  26.63  26.63  3.279e+05          0.0   

   Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  Adj. Volume  
0          1.0      0.423      0.424     0.423       0.423    1.173e+08  
1          1.0      0.403      0.403     0.401       0.401    4.397e+07  
2          1.0      0.373      0.373     0.371       0.371    2.643e+07  
3          1.0      0.380      0.382     0.380       0.380    2.161e+07  
4          1.0      0.392      0.393     0.392       0.392    1.836e+07  

In [48]:
# Create Pivot of the dataframe
closes = s4p.pivot(index='Date', columns='Symbol',
                   values='Adj. Close')

In [49]:
closes[:3]

Symbol       AAPL  MSFT
Date                   
1980-12-12  0.423   NaN
1980-12-15  0.401   NaN
1980-12-16  0.371   NaN

In [50]:
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
1980-12-12  AAPL        0.423
1980-12-15  AAPL        0.401
1980-12-16  AAPL        0.371
1980-12-17  AAPL        0.380
                       ...   
2018-03-26  AAPL      172.770
            MSFT       93.780
2018-03-27  AAPL      168.340
            MSFT       89.470
Length: 17476, dtype: float64

In [51]:
stackedCloses.loc['2012-01-03', 'AAPL']

52.848786580038

In [54]:
stackedCloses.loc['2012-01-03']

Date        Symbol
2012-01-03  AAPL      52.849
            MSFT      22.792
dtype: float64

In [55]:
stackedCloses.loc[:, 'MSFT']

Date
1986-03-13     0.065
1986-03-14     0.067
1986-03-17     0.068
1986-03-18     0.066
               ...  
2018-03-22    89.790
2018-03-23    87.180
2018-03-26    93.780
2018-03-27    89.470
Length: 8076, dtype: float64

In [56]:
unstackedCloses = stackedCloses.unstack()

In [57]:
unstackedCloses[:3]

Symbol       AAPL  MSFT
Date                   
1980-12-12  0.423   NaN
1980-12-15  0.401   NaN
1980-12-16  0.371   NaN

In [59]:
melted = pd.melt(s4p, id_vars=['Date', 'Symbol'])


In [60]:
melted[:5]

        Date Symbol variable  value
0 1980-12-12   AAPL     Open  28.75
1 1980-12-15   AAPL     Open  27.38
2 1980-12-16   AAPL     Open  25.37
3 1980-12-17   AAPL     Open  25.87
4 1980-12-18   AAPL     Open  26.63

In [62]:
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

             Date Symbol     variable      value
14343  2012-01-03   MSFT         Open  2.655e+01
31819  2012-01-03   MSFT         High  2.696e+01
49295  2012-01-03   MSFT          Low  2.639e+01
66771  2012-01-03   MSFT        Close  2.677e+01
...           ...    ...          ...        ...
154151 2012-01-03   MSFT    Adj. High  2.296e+01
171627 2012-01-03   MSFT     Adj. Low  2.247e+01
189103 2012-01-03   MSFT   Adj. Close  2.279e+01
206579 2012-01-03   MSFT  Adj. Volume  6.473e+07

[12 rows x 4 columns]

In [81]:
s4g = combined[['Symbol', 'Adj. Close']].reset_index()
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)

In [82]:
s4g[:5]

        Date  Year  Month Symbol  Adj. Close
0 1980-12-12  1980     12   AAPL       0.423
1 1980-12-15  1980     12   AAPL       0.401
2 1980-12-16  1980     12   AAPL       0.371
3 1980-12-17  1980     12   AAPL       0.380
4 1980-12-18  1980     12   AAPL       0.392

In [67]:
s4g.groupby('Symbol')

In [68]:
grouped = s4g.groupby('Symbol')
type(grouped.groups)

dict

In [69]:
grouped.groups

{'AAPL': Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                 9,
             ...
             17456, 17458, 17460, 17463, 17465, 17467, 17469, 17471, 17472,
             17475],
            dtype='int64', length=9400),
 'MSFT': Int64Index([ 1326,  1327,  1329,  1332,  1333,  1335,  1337,  1339,  1341,
              1344,
             ...
             17457, 17459, 17461, 17462, 17464, 17466, 17468, 17470, 17473,
             17474],
            dtype='int64', length=8076)}

In [70]:
len(grouped), grouped.ngroups

(2, 2)

In [71]:
def print_groups (groupobject):
    for name, group in groupobject:
        print(name)
        print(group.head())

In [72]:
print_groups(grouped)

AAPL
        Date  Year  Month[:5] Symbol  Adj. Close
0 1980-12-12  1980         12   AAPL       0.423
1 1980-12-15  1980         12   AAPL       0.401
2 1980-12-16  1980         12   AAPL       0.371
3 1980-12-17  1980         12   AAPL       0.380
4 1980-12-18  1980         12   AAPL       0.392
MSFT
           Date  Year  Month[:5] Symbol  Adj. Close
1326 1986-03-13  1986          3   MSFT       0.065
1327 1986-03-14  1986          3   MSFT       0.067
1329 1986-03-17  1986          3   MSFT       0.068
1332 1986-03-18  1986          3   MSFT       0.066
1333 1986-03-19  1986          3   MSFT       0.065


In [73]:
grouped.size()

Symbol
AAPL    9400
MSFT    8076
dtype: int64

In [75]:
grouped.get_group('MSFT')

            Date  Year  Month[:5] Symbol  Adj. Close
1326  1986-03-13  1986          3   MSFT       0.065
1327  1986-03-14  1986          3   MSFT       0.067
1329  1986-03-17  1986          3   MSFT       0.068
1332  1986-03-18  1986          3   MSFT       0.066
...          ...   ...        ...    ...         ...
17468 2018-03-22  2018          3   MSFT      89.790
17470 2018-03-23  2018          3   MSFT      87.180
17473 2018-03-26  2018          3   MSFT      93.780
17474 2018-03-27  2018          3   MSFT      89.470

[8076 rows x 5 columns]

In [83]:
mcg = s4g.groupby(['Symbol', 'Year','Month'])




In [84]:
print_groups(mcg)

('AAPL', 1980, 12)
        Date  Year  Month Symbol  Adj. Close
0 1980-12-12  1980     12   AAPL       0.423
1 1980-12-15  1980     12   AAPL       0.401
2 1980-12-16  1980     12   AAPL       0.371
3 1980-12-17  1980     12   AAPL       0.380
4 1980-12-18  1980     12   AAPL       0.392
('AAPL', 1981, 1)
         Date  Year  Month Symbol  Adj. Close
13 1981-01-02  1981      1   AAPL       0.507
14 1981-01-05  1981      1   AAPL       0.496
15 1981-01-06  1981      1   AAPL       0.474
16 1981-01-07  1981      1   AAPL       0.454
17 1981-01-08  1981      1   AAPL       0.445
('AAPL', 1981, 2)
         Date  Year  Month Symbol  Adj. Close
34 1981-02-02  1981      2   AAPL       0.392
35 1981-02-03  1981      2   AAPL       0.406
36 1981-02-04  1981      2   AAPL       0.421
37 1981-02-05  1981      2   AAPL       0.421
38 1981-02-06  1981      2   AAPL       0.423
('AAPL', 1981, 3)
         Date  Year  Month Symbol  Adj. Close
53 1981-03-02  1981      3   AAPL       0.392
54 1981-03-03

711 1983-10-07  1983     10   AAPL       0.300
('AAPL', 1983, 11)
          Date  Year  Month Symbol  Adj. Close
728 1983-11-01  1983     11   AAPL       0.338
729 1983-11-02  1983     11   AAPL       0.346
730 1983-11-03  1983     11   AAPL       0.322
731 1983-11-04  1983     11   AAPL       0.311
732 1983-11-07  1983     11   AAPL       0.309
('AAPL', 1983, 12)
          Date  Year  Month Symbol  Adj. Close
749 1983-12-01  1983     12   AAPL       0.298
750 1983-12-02  1983     12   AAPL       0.292
751 1983-12-05  1983     12   AAPL       0.300
752 1983-12-06  1983     12   AAPL       0.301
753 1983-12-07  1983     12   AAPL       0.309
('AAPL', 1984, 1)
          Date  Year  Month Symbol  Adj. Close
770 1984-01-03  1984      1   AAPL       0.377
771 1984-01-04  1984      1   AAPL       0.410
772 1984-01-05  1984      1   AAPL       0.415
773 1984-01-06  1984      1   AAPL       0.408
774 1984-01-09  1984      1   AAPL       0.386
('AAPL', 1984, 2)
          Date  Year  Month Symbo

2163 1987-11-06  1987     11   AAPL       1.110
('AAPL', 1987, 12)
           Date  Year  Month Symbol  Adj. Close
2196 1987-12-01  1987     12   AAPL       0.980
2197 1987-12-02  1987     12   AAPL       0.958
2199 1987-12-03  1987     12   AAPL       0.899
2202 1987-12-04  1987     12   AAPL       0.906
2204 1987-12-07  1987     12   AAPL       0.973
('AAPL', 1988, 1)
           Date  Year  Month Symbol  Adj. Close
2239 1988-01-04  1988      1   AAPL       1.319
2241 1988-01-05  1988      1   AAPL       1.315
2243 1988-01-06  1988      1   AAPL       1.289
2246 1988-01-07  1988      1   AAPL       1.312
2247 1988-01-08  1988      1   AAPL       1.179
('AAPL', 1988, 2)
           Date  Year  Month Symbol  Adj. Close
2280 1988-02-01  1988      2   AAPL       1.230
2282 1988-02-02  1988      2   AAPL       1.216
2283 1988-02-03  1988      2   AAPL       1.164
2286 1988-02-04  1988      2   AAPL       1.172
2287 1988-02-05  1988      2   AAPL       1.139
('AAPL', 1988, 3)
           Date

('AAPL', 1991, 10)
           Date  Year  Month Symbol  Adj. Close
4133 1991-10-01  1991     10   AAPL       1.553
4135 1991-10-02  1991     10   AAPL       1.523
4137 1991-10-03  1991     10   AAPL       1.462
4140 1991-10-04  1991     10   AAPL       1.477
4142 1991-10-07  1991     10   AAPL       1.473
('AAPL', 1991, 11)
           Date  Year  Month Symbol  Adj. Close
4180 1991-11-01  1991     11   AAPL       1.561
4181 1991-11-04  1991     11   AAPL       1.523
4184 1991-11-05  1991     11   AAPL       1.492
4185 1991-11-06  1991     11   AAPL       1.469
4187 1991-11-07  1991     11   AAPL       1.523
('AAPL', 1991, 12)
           Date  Year  Month Symbol  Adj. Close
4220 1991-12-02  1991     12   AAPL       1.588
4222 1991-12-03  1991     12   AAPL       1.549
4224 1991-12-04  1991     12   AAPL       1.549
4226 1991-12-05  1991     12   AAPL       1.534
4228 1991-12-06  1991     12   AAPL       1.496
('AAPL', 1992, 1)
           Date  Year  Month Symbol  Adj. Close
4262 1992-01-

6375 1996-03-07  1996      3   AAPL       0.829
('AAPL', 1996, 4)
           Date  Year  Month Symbol  Adj. Close
6409 1996-04-01  1996      4   AAPL       0.819
6412 1996-04-02  1996      4   AAPL       0.803
6413 1996-04-03  1996      4   AAPL       0.789
6415 1996-04-04  1996      4   AAPL       0.775
6418 1996-04-08  1996      4   AAPL       0.783
('AAPL', 1996, 5)
           Date  Year  Month Symbol  Adj. Close
6451 1996-05-01  1996      5   AAPL       0.783
6454 1996-05-02  1996      5   AAPL       0.763
6456 1996-05-03  1996      5   AAPL       0.767
6457 1996-05-06  1996      5   AAPL       0.823
6459 1996-05-07  1996      5   AAPL       0.863
('AAPL', 1996, 6)
           Date  Year  Month Symbol  Adj. Close
6495 1996-06-03  1996      6   AAPL       0.795
6497 1996-06-04  1996      6   AAPL       0.777
6500 1996-06-05  1996      6   AAPL       0.807
6501 1996-06-06  1996      6   AAPL       0.779
6503 1996-06-07  1996      6   AAPL       0.783
('AAPL', 1996, 7)
           Date 

...

5490 1994-06-07  1994      6   MSFT       2.231
('MSFT', 1994, 7)
           Date  Year  Month Symbol  Adj. Close
5525 1994-07-01  1994      7   MSFT       2.091
5528 1994-07-05  1994      7   MSFT       2.044
5530 1994-07-06  1994      7   MSFT       2.010
5531 1994-07-07  1994      7   MSFT       2.096
5533 1994-07-08  1994      7   MSFT       2.059
('MSFT', 1994, 8)
           Date  Year  Month Symbol  Adj. Close
5565 1994-08-01  1994      8   MSFT       2.221
5567 1994-08-02  1994      8   MSFT       2.197
5569 1994-08-03  1994      8   MSFT       2.221
5571 1994-08-04  1994      8   MSFT       2.195
5573 1994-08-05  1994      8   MSFT       2.184
('MSFT', 1994, 9)
           Date  Year  Month Symbol  Adj. Close
5611 1994-09-01  1994      9   MSFT       2.335
5613 1994-09-02  1994      9   MSFT       2.330
5615 1994-09-06  1994      9   MSFT       2.330
5617 1994-09-07  1994      9   MSFT       2.367
5619 1994-09-08  1994      9   MSFT       2.411
('MSFT', 1994, 10)
           Date

('MSFT', 1999, 1)
           Date  Year  Month Symbol  Adj. Close
7801 1999-01-04  1999      1   MSFT      23.466
7804 1999-01-05  1999      1   MSFT      24.381
7806 1999-01-06  1999      1   MSFT      25.171
7808 1999-01-07  1999      1   MSFT      25.047
7810 1999-01-08  1999      1   MSFT      24.943
('MSFT', 1999, 2)
           Date  Year  Month Symbol  Adj. Close
7839 1999-02-01  1999      2   MSFT      28.781
7842 1999-02-02  1999      2   MSFT      27.897
7843 1999-02-03  1999      2   MSFT      27.761
7846 1999-02-04  1999      2   MSFT      26.471
7848 1999-02-05  1999      2   MSFT      26.628
('MSFT', 1999, 3)
           Date  Year  Month Symbol  Adj. Close
7877 1999-03-01  1999      3   MSFT      25.255
7879 1999-03-02  1999      3   MSFT      24.724
7882 1999-03-03  1999      3   MSFT      24.902
7883 1999-03-04  1999      3   MSFT      25.338
7885 1999-03-05  1999      3   MSFT      25.786
('MSFT', 1999, 4)
           Date  Year  Month Symbol  Adj. Close
7923 1999-04-01 

10569 2004-07-08  2004      7   MSFT      18.561
('MSFT', 2004, 8)
            Date  Year  Month Symbol  Adj. Close
10604 2004-08-02  2004      8   MSFT      19.152
10605 2004-08-03  2004      8   MSFT      18.850
10607 2004-08-04  2004      8   MSFT      18.843
10609 2004-08-05  2004      8   MSFT      18.487
10612 2004-08-06  2004      8   MSFT      18.226
('MSFT', 2004, 9)
            Date  Year  Month Symbol  Adj. Close
10647 2004-09-01  2004      9   MSFT      18.447
10649 2004-09-02  2004      9   MSFT      18.602
10652 2004-09-03  2004      9   MSFT      18.259
10653 2004-09-07  2004      9   MSFT      18.427
10655 2004-09-08  2004      9   MSFT      18.360
('MSFT', 2004, 10)
            Date  Year  Month Symbol  Adj. Close
10689 2004-10-01  2004     10   MSFT      19.027
10692 2004-10-04  2004     10   MSFT      18.939
10693 2004-10-05  2004     10   MSFT      19.114
10695 2004-10-06  2004     10   MSFT      19.215
10697 2004-10-07  2004     10   MSFT      18.973
('MSFT', 2004,

13507 2010-05-07  2010      5   MSFT      23.032
('MSFT', 2010, 6)
            Date  Year  Month Symbol  Adj. Close
13540 2010-06-01  2010      6   MSFT      21.234
13542 2010-06-02  2010      6   MSFT      21.701
13544 2010-06-03  2010      6   MSFT      22.029
13546 2010-06-04  2010      6   MSFT      21.152
13548 2010-06-07  2010      6   MSFT      20.742
('MSFT', 2010, 7)
            Date  Year  Month Symbol  Adj. Close
13583 2010-07-01  2010      7   MSFT      18.995
13585 2010-07-02  2010      7   MSFT      19.085
13587 2010-07-06  2010      7   MSFT      19.536
13590 2010-07-07  2010      7   MSFT      19.930
13591 2010-07-08  2010      7   MSFT      20.019
('MSFT', 2010, 8)
            Date  Year  Month Symbol  Adj. Close
13625 2010-08-02  2010      8   MSFT      21.595
13628 2010-08-03  2010      8   MSFT      21.455
13630 2010-08-04  2010      8   MSFT      21.102
13631 2010-08-05  2010      8   MSFT      20.807
13633 2010-08-06  2010      8   MSFT      20.955
('MSFT', 2010, 

16528 2016-05-06  2016      5   MSFT      48.320
('MSFT', 2016, 6)
            Date  Year  Month Symbol  Adj. Close
16562 2016-06-01  2016      6   MSFT      51.041
16564 2016-06-02  2016      6   MSFT      50.683
16565 2016-06-03  2016      6   MSFT      50.017
16568 2016-06-06  2016      6   MSFT      50.345
16569 2016-06-07  2016      6   MSFT      50.316
('MSFT', 2016, 7)
            Date  Year  Month Symbol  Adj. Close
16606 2016-07-01  2016      7   MSFT      49.408
16608 2016-07-05  2016      7   MSFT      49.418
16609 2016-07-06  2016      7   MSFT      49.621
16611 2016-07-07  2016      7   MSFT      49.621
16613 2016-07-08  2016      7   MSFT      50.509
('MSFT', 2016, 8)
            Date  Year  Month Symbol  Adj. Close
16646 2016-08-01  2016      8   MSFT      54.643
16648 2016-08-02  2016      8   MSFT      54.643
16650 2016-08-03  2016      8   MSFT      55.020
16651 2016-08-04  2016      8   MSFT      55.425
16654 2016-08-05  2016      8   MSFT      55.976
('MSFT', 2016, 

In [88]:
mi = s4g.set_index(['Symbol', 'Year', 'Month'])
mi

                        Date  Adj. Close
Symbol Year Month                       
AAPL   1980 12    1980-12-12       0.423
            12    1980-12-15       0.401
            12    1980-12-16       0.371
            12    1980-12-17       0.380
...                      ...         ...
       2018 3     2018-03-26     172.770
MSFT   2018 3     2018-03-26      93.780
            3     2018-03-27      89.470
AAPL   2018 3     2018-03-27     168.340

[17476 rows x 2 columns]

In [89]:
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

AAPL
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1980 12    1980-12-12       0.423
            12    1980-12-15       0.401
            12    1980-12-16       0.371
            12    1980-12-17       0.380
            12    1980-12-18       0.392
MSFT
                        Date  Adj. Close
Symbol Year Month                       
MSFT   1986 3     1986-03-13       0.065
            3     1986-03-14       0.067
            3     1986-03-17       0.068
            3     1986-03-18       0.066
            3     1986-03-19       0.065


In [90]:
#Grouping by multiple levels can be performed by passing multiple levels to
#.groupby(). Also, if the MultiIndex has names specified, then those names can be
#used instead of the integers for the levels. The following command groups the three
#levels of the MultiIndex by using their names:

mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)


('AAPL', 1980, 12)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1980 12    1980-12-12       0.423
            12    1980-12-15       0.401
            12    1980-12-16       0.371
            12    1980-12-17       0.380
            12    1980-12-18       0.392
('AAPL', 1981, 1)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1981 1     1981-01-02       0.507
            1     1981-01-05       0.496
            1     1981-01-06       0.474
            1     1981-01-07       0.454
            1     1981-01-08       0.445
('AAPL', 1981, 2)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1981 2     1981-02-02       0.392
            2     1981-02-03       0.406
            2     1981-02-04       0.421
            2     1981-02-05       0.421
            2     1981-02-06       0.423
('AAPL', 1981, 3)
                        Date  Adj. Close
Symbol Year Month        

                        Date  Adj. Close
Symbol Year Month                       
AAPL   1983 12    1983-12-01       0.298
            12    1983-12-02       0.292
            12    1983-12-05       0.300
            12    1983-12-06       0.301
            12    1983-12-07       0.309
('AAPL', 1984, 1)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1984 1     1984-01-03       0.377
            1     1984-01-04       0.410
            1     1984-01-05       0.415
            1     1984-01-06       0.408
            1     1984-01-09       0.386
('AAPL', 1984, 2)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1984 2     1984-02-01       0.362
            2     1984-02-02       0.366
            2     1984-02-03       0.360
            2     1984-02-06       0.342
            2     1984-02-07       0.355
('AAPL', 1984, 3)
                        Date  Adj. Close
Symbol Year Month                       
AAP

                        Date  Adj. Close
Symbol Year Month                       
AAPL   1987 4     1987-04-01       0.981
            4     1987-04-02       1.055
            4     1987-04-03       1.055
            4     1987-04-06       1.029
            4     1987-04-07       0.996
('AAPL', 1987, 5)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1987 5     1987-05-01       1.176
            5     1987-05-04       1.173
            5     1987-05-05       1.180
            5     1987-05-06       1.176
            5     1987-05-07       1.180
('AAPL', 1987, 6)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1987 6     1987-06-01       1.143
            6     1987-06-02       1.136
            6     1987-06-03       1.143
            6     1987-06-04       1.154
            6     1987-06-05       1.143
('AAPL', 1987, 7)
                        Date  Adj. Close
Symbol Year Month                       
AAP

                        Date  Adj. Close
Symbol Year Month                       
AAPL   1991 6     1991-06-03       1.504
            6     1991-06-04       1.500
            6     1991-06-05       1.466
            6     1991-06-06       1.424
            6     1991-06-07       1.409
('AAPL', 1991, 7)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1991 7     1991-07-01       1.298
            7     1991-07-02       1.290
            7     1991-07-03       1.317
            7     1991-07-05       1.393
            7     1991-07-08       1.428
('AAPL', 1991, 8)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1991 8     1991-08-01       1.500
            8     1991-08-02       1.527
            8     1991-08-05       1.481
            8     1991-08-06       1.512
            8     1991-08-07       1.538
('AAPL', 1991, 9)
                        Date  Adj. Close
Symbol Year Month                       
AAP

                        Date  Adj. Close
Symbol Year Month                       
AAPL   1995 9     1995-09-01       1.375
            9     1995-09-05       1.393
            9     1995-09-06       1.401
            9     1995-09-07       1.433
            9     1995-09-08       1.433
('AAPL', 1995, 10)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1995 10    1995-10-02       1.205
            10    1995-10-03       1.205
            10    1995-10-04       1.165
            10    1995-10-05       1.169
            10    1995-10-06       1.143
('AAPL', 1995, 11)
                        Date  Adj. Close
Symbol Year Month                       
AAPL   1995 11    1995-11-01       1.173
            11    1995-11-02       1.173
            11    1995-11-03       1.169
            11    1995-11-06       1.221
            11    1995-11-07       1.269
('AAPL', 1995, 12)
                        Date  Adj. Close
Symbol Year Month                       


...

                        Date  Adj. Close
Symbol Year Month                       
MSFT   1997 4     1997-04-01       7.759
            4     1997-04-02       7.655
            4     1997-04-03       7.915
            4     1997-04-04       7.838
            4     1997-04-07       7.977
('MSFT', 1997, 5)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   1997 5     1997-05-01      10.069
            5     1997-05-02      10.048
            5     1997-05-05      10.001
            5     1997-05-06       9.762
            5     1997-05-07       9.611
('MSFT', 1997, 6)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   1997 6     1997-06-02      10.349
            6     1997-06-03      10.048
            6     1997-06-04       9.918
            6     1997-06-05      10.032
            6     1997-06-06      10.323
('MSFT', 1997, 7)
                        Date  Adj. Close
Symbol Year Month                       
MSF

                        Date  Adj. Close
Symbol Year Month                       
MSFT   2001 6     2001-06-01      23.412
            6     2001-06-04      23.559
            6     2001-06-05      24.165
            6     2001-06-06      24.085
            6     2001-06-07      24.524
('MSFT', 2001, 7)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2001 7     2001-07-02      23.499
            7     2001-07-03      23.456
            7     2001-07-05      22.803
            7     2001-07-06      21.988
            7     2001-07-09      21.865
('MSFT', 2001, 8)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2001 8     2001-08-01      22.124
            8     2001-08-02      22.450
            8     2001-08-03      22.264
            8     2001-08-06      22.011
            8     2001-08-07      22.084
('MSFT', 2001, 9)
                        Date  Adj. Close
Symbol Year Month                       
MSF

            8     2006-08-07      18.500
('MSFT', 2006, 9)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2006 9     2006-09-01      19.810
            9     2006-09-05      19.633
            9     2006-09-06      19.633
            9     2006-09-07      19.495
            9     2006-09-08      19.626
('MSFT', 2006, 10)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2006 10    2006-10-02      20.975
            10    2006-10-03      20.983
            10    2006-10-04      21.420
            10    2006-10-05      21.404
            10    2006-10-06      21.366
('MSFT', 2006, 11)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2006 11    2006-11-01      22.087
            11    2006-11-02      22.056
            11    2006-11-03      22.025
            11    2006-11-06      22.110
            11    2006-11-07      22.194
('MSFT', 2006, 12)
                       

            9     2010-09-08      19.729
('MSFT', 2010, 10)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2010 10    2010-10-01      20.100
            10    2010-10-04      19.713
            10    2010-10-05      20.076
            10    2010-10-06      20.142
            10    2010-10-07      20.224
('MSFT', 2010, 11)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2010 11    2010-11-01      22.219
            11    2010-11-02      22.582
            11    2010-11-03      22.285
            11    2010-11-04      22.376
            11    2010-11-05      22.137
('MSFT', 2010, 12)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2010 12    2010-12-01      21.602
            12    2010-12-02      22.307
            12    2010-12-03      22.415
            12    2010-12-06      22.266
            12    2010-12-07      22.291
('MSFT', 2011, 1)
                       

                        Date  Adj. Close
Symbol Year Month                       
MSFT   2015 8     2015-08-03      43.984
            8     2015-08-04      44.670
            8     2015-08-05      44.708
            8     2015-08-06      43.805
            8     2015-08-07      43.918
('MSFT', 2015, 9)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2015 9     2015-09-01      39.553
            9     2015-09-02      41.010
            9     2015-09-03      41.142
            9     2015-09-04      40.300
            9     2015-09-08      41.511
('MSFT', 2015, 10)
                        Date  Adj. Close
Symbol Year Month                       
MSFT   2015 10    2015-10-01      42.192
            10    2015-10-02      43.100
            10    2015-10-05      44.102
            10    2015-10-06      44.216
            10    2015-10-07      44.263
('MSFT', 2015, 11)
                        Date  Adj. Close
Symbol Year Month                       
M

In [91]:
mig_l12.agg(np.mean)

                   Adj. Close
Symbol Year Month            
AAPL   1980 12          0.448
       1981 1           0.465
            2           0.388
            3           0.365
...                       ...
MSFT   2017 12         84.758
       2018 1          90.074
            2          91.413
            3          93.169

[833 rows x 1 columns]

In [92]:
#Group by and Apply mean value of adj Close
s4g.groupby(['Symbol', 'Year', 'Month'],
            as_index=False).agg(np.mean)[:5]
            

  Symbol  Year  Month  Adj. Close
0   AAPL  1980     12       0.448
1   AAPL  1981      1       0.465
2   AAPL  1981      2       0.388
3   AAPL  1981      3       0.365
4   AAPL  1981      4       0.400

In [93]:
mig_l12.agg([np.mean, np.std])


                  Adj. Close       
                        mean    std
Symbol Year Month                  
AAPL   1980 12         0.448  0.057
       1981 1          0.465  0.021
            2          0.388  0.021
            3          0.365  0.023
...                      ...    ...
MSFT   2017 12        84.758  1.576
       2018 1         90.074  2.628
            2         91.413  2.569
            3         93.169  2.282

[833 rows x 2 columns]

## Time Series

In [103]:
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 8)
pd.set_option('precision', 7)
import datetime
from datetime import datetime

In [104]:
dates = [datetime(2014, 8, 1), datetime(2014, 8, 2)]
dti = pd.DatetimeIndex(dates)
dti

DatetimeIndex(['2014-08-01', '2014-08-02'], dtype='datetime64[ns]', freq=None)

In [105]:
np.random.seed(123456)
ts = pd.Series(np.random.randn(2), dates)
type(ts.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [106]:
# Select specific month to specific date of the month
msft['2012-2':'2012-02-09'][:5]

           Symbol     Open    High    Low  Close     ...        Adj. Open  \
Date                                                 ...                    
2012-02-01   MSFT  29.7900  30.050  29.76  29.89     ...       25.3682457   
2012-02-02   MSFT  29.9000  30.170  29.71  29.95     ...       25.4619183   
2012-02-03   MSFT  30.1401  30.400  30.09  30.24     ...       25.6663801   
2012-02-06   MSFT  30.0400  30.220  29.97  30.20     ...       25.5811380   
2012-02-07   MSFT  30.1500  30.485  30.05  30.35     ...       25.6748106   

             Adj. High    Adj. Low  Adj. Close  Adj. Volume  
Date                                                         
2012-02-01  25.5896537  25.3426986  25.4534026   67409900.0  
2012-02-02  25.6918420  25.3001202  25.5044968   52223300.0  
2012-02-03  25.8877029  25.6237164  25.7514518   41838500.0  
2012-02-06  25.7344204  25.5215281  25.7173890   28039700.0  
2012-02-07  25.9600863  25.5896537  25.8451244   39242400.0  

[5 rows x 13 columns]

## Creating time-series with specific frequencies

In [107]:
bymin = pd.Series(np.arange(0, 90*60*24),
                  pd.date_range('2014-08-01',
                                '2014-10-29 23:59:00',
                                freq='T'))
bymin
                  

2014-08-01 00:00:00         0
2014-08-01 00:01:00         1
2014-08-01 00:02:00         2
2014-08-01 00:03:00         3
                        ...  
2014-10-29 23:56:00    129596
2014-10-29 23:57:00    129597
2014-10-29 23:58:00    129598
2014-10-29 23:59:00    129599
Freq: T, Length: 129600, dtype: int32

In [108]:
bymin['2014-08-01 12:30':'2014-08-01 12:59']

2014-08-01 12:30:00    750
2014-08-01 12:31:00    751
2014-08-01 12:32:00    752
2014-08-01 12:33:00    753
                      ... 
2014-08-01 12:56:00    776
2014-08-01 12:57:00    777
2014-08-01 12:58:00    778
2014-08-01 12:59:00    779
Freq: T, Length: 30, dtype: int32